In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir( os.path.join('..', '..', 'notebook_format') )
from formats import load_style
load_style()

In [2]:
os.chdir(path)
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8, 6 # change default figure size
plt.rcParams['font.size'] = 12 # and font size

# 1. magic to print version
# 2. magic so that the notebook will reload external python modules
%load_ext watermark
%load_ext autoreload 
%autoreload 2
%load_ext cython

# from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,matplotlib,sklearn,scipy

Ethen 2017-01-21 20:22:59 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.3
pandas 0.18.1
matplotlib 1.5.1
sklearn 0.18
scipy 0.18.1


In [3]:
%%cython -a
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
import numpy as np
from libc.string cimport memcpy
from libc.stdlib cimport malloc, free
from scipy.linalg cimport cython_blas

cdef inline void outer(int* m, int* n, double* alpha, double* x, int* incx,
                       double* y, int* incy, double* A, int* lda) nogil:
    """
    m : (input)
        the number of rows of matrix A. M >= 0

    n : (input)
        the number of columns of matrix A. N >= 0

    alpha : (input)
        the scalar alpha

    x : (input)
         m element vector x

    incx : (input)
        the increment for the elements of x

    y : (input)
        n element vector y

    incy : (input)
        the increment for the elements of y

    A (input/output)
        Before entry, the leading m by n part of the array
        A  must contain the matrix  of coefficients. On
        exit, A is overwritten by the updated matrix.

    lda (input)
        specifies the leading dimension of A, simply the row number   
    """
    cython_blas.dger(m, n, alpha, x, incx, y, incy, A, lda)
    

cdef:
    int i, one = 1
    double alpha = 1.0
    double* A
    double[:, :] X = np.array([[9, 4], [3, 1], [5, 2]], dtype = np.float64)
    int n_factors = X.shape[1]
    double[:, :] end_result = np.zeros((n_factors, n_factors), dtype = np.float64)


A = <double*> malloc(sizeof(double) * n_factors * n_factors)
memcpy(A, &end_result[0, 0], sizeof(double) * n_factors * n_factors)
memcpy(&end_result[0, 0], A, sizeof(double) * n_factors * n_factors)
print(np.asarray(end_result))

for i in range(X.shape[0]):
    print(np.asarray(X[i]))
    outer(&n_factors, &n_factors, &alpha, &X[i, 0], &one, &X[i, 0], &one, A, &n_factors)
    memcpy(&end_result[0, 0], A, sizeof(double) * n_factors * n_factors)
    print(np.asarray(end_result))

free(A)

[[ 0.  0.]
 [ 0.  0.]]
[ 9.  4.]
[[ 81.  36.]
 [ 36.  16.]]
[ 3.  1.]
[[ 90.  39.]
 [ 39.  17.]]
[ 5.  2.]
[[ 115.   49.]
 [  49.   21.]]


In [10]:
reg = 0.1
n_factors = 20
n_threads = 0

from movielens import create_train_test
from scipy.sparse import csr_matrix
file_dir = 'ml-100k'
train, test = create_train_test(file_dir)
R_ui_train = csr_matrix(train)
R_ui_test = csr_matrix(test)
Cui = R_ui_train
Ciu = R_ui_train.T

n_users, n_items = Cui.shape
X = np.random.rand(n_users, n_factors)
Y = np.random.rand(n_items, n_factors)

In [15]:
%%cython -a
# cython: boundscheck = False
# cython: wraparound = False

cimport cython
import numpy as np
from libc.string cimport memcpy
from libc.stdlib cimport malloc, free
from cython.parallel import parallel, prange
from scipy.linalg cimport cython_blas, cython_lapack

cdef inline void axpy(int* n, double* alpha, double* x,
                      int* incx, double* y, int* incy) nogil:
    """
    compute y := alpha * x + y where alpha is a scalar and
    x and y are n-vectors
    
    Parameters
    ----------
    n : (input)
        number of elements in the vector

    alpha : (input)
        the scalar alpha

    x : (input)
        the vector x

    incx : (input)
        the increment for the elements of x, should be 1 unless
        we want to skip elements

    y : (input/output)
        the vector y. On exit, y is overwritten by the updated vector y

    incy : (input)
        the increment for the elements of y
    
    Reference
    ---------
    https://docs.oracle.com/cd/E19422-01/819-3691/daxpy.html
    """
    cython_blas.daxpy(n, alpha, x, incx, y, incy)
    

cdef inline void outer(int* m, int* n, double* alpha, double* x, int* incx,
                       double* y, int* incy, double* A, int* lda) nogil:
    """
    m : (input)
        the number of rows of matrix A. M >= 0

    n : (input)
        the number of columns of matrix A. N >= 0

    alpha : (input)
        the scalar alpha

    x : (input)
         m element vector x

    incx : (input)
        the increment for the elements of x

    y : (input)
        n element vector y

    incy : (input)
        the increment for the elements of y

    A (input/output)
        Before entry, the leading m by n part of the array
        A  must contain the matrix  of coefficients. On
        exit, A is overwritten by the updated matrix.

    lda (input)
        specifies the leading dimension of A, simply the row number

    Reference
    ---------
    https://docs.oracle.com/cd/E19422-01/819-3691/dger.html
    http://stackoverflow.com/questions/19845071/outer-product-using-cblas 
    """
    cython_blas.dger(m, n, alpha, x, incx, y, incy, A, lda)

    
cdef inline void posv(char* uplo, int* n, int* nrhs, double* A, 
                      int* lda, double* b, int* ldb, int* info) nogil:
    """
    computes the solution of linear equations A * X = B

    The Cholesky decomposition is used to factor A as
    A = U**T* U,  if UPLO = 'U', or A = L * L**T,  if UPLO = 'L',
    where U is an upper triangular matrix and L is a lower tri-
    angular matrix. The  factored form of A is then used to
    solve the system of equations A * X = B
     
    Parameters
    ----------
    uplo : (input)
        'U':  Upper triangle of A is stored
        'L':  Lower triangle of A is stored
    
    n : (input) 
        number of linear equations, i.e., the order of the matrix A

    nrhs : (input)
        number of right hand sides, i.e., number
        of columns of the matrix b, should be 1 unless solve for
        multiple b

    A : (input/output)
        the matrix A. On exit, if INFO = 0, the factor U or L 
        from the Cholesky factorization A = U**T*U or A = L*L**T

    lda (input)
        leading dimension of the array A, simply the row number
        if we're using the whole array

    b : (input/output)
        the right hand side matrix b. On exit, if INFO = 0, 
        the solution matrix X

    ldb (input)
        leading dimension of b
     
    info: (output)
        = 0: successful exit
        < 0: if INFO = -i, the i-th argument had an illegal value
        > 0: if INFO = i, the leading minor of order i of
             A  is  not positive definite, so the factorization
             could not be completed, and the solution  has  not
             been computed.

    Reference
    ---------
    https://docs.oracle.com/cd/E19422-01/819-3691/dposv.html
    """
    cython_lapack.dposv(uplo, n, nrhs, A, lda, b, ldb, info)
    

cdef inline void gesv(int* n, int* nrhs, double* a, int* lda, 
                      int* pivot, double* b, int* ldb, int* info) nogil:
    """
    computes the solution of linear equations A * X = B;
    The LU decomposition with partial pivoting and row
    interchanges is used to factor A as A = P * L * U,
    where P is a permutation matrix, L is unit lower triangular,
    and U is upper triangular. The factored form of A is then
    used to solve the system of equations A * X = B

    n : (input)
        number of linear equations
    
    nrhs : (input)
        number of right hand sides

    A (input/output)
        On entry, the N-by-N coefficient matrix A. On
        exit, the factors L and U from the factorization A
        = P * L * U; the unit diagonal elements of L are not stored

    lda : (input)
        leading dimension of the array A, simply the row number
        if we're using the whole array

    IPIVOT (output)
        The pivot  indices  that  define  the  permutation
        matrix  P;  row  i  of the matrix was interchanged
        with row IPIVOT(i).

    b (input/output)
        On entry, the N-by-NRHS matrix of right hand  side
        matrix B. On exit, if INFO = 0, the N-by-NRHS
        solution matrix X

    ldb : (input)
        leading dimension of b
     
    info : (output)
        = 0 : successful exit
        < 0 : if INFO = -i, the i-th argument had an illegal value
        > 0 : if INFO = i, the factor U is exactly
              singular, so the solution could not be computed

    Reference
    ---------
    https://docs.oracle.com/cd/E19422-01/819-3691/dgesv.html
    """
    cython_lapack.dgesv(n, nrhs, a, lda, pivot, b, ldb, info)


cpdef void _als_step(Cui, double[:, :] X, double[:, :] Y, double reg, int n_threads):    
    cdef:
        double confidence, temp
        int u, i, j, index, error, one = 1
        double[:] data = Cui.data
        int[:] indptr = Cui.indptr, indices = Cui.indices
        int n_users = X.shape[0], n_factors = X.shape[1]
        
        # we need initialize a new A and b for every user,
        #  but cdef statement not allowed in parallel loop,
        # so use pointers to access the memory address instead
        double* b
        double* A
        double[:, :] YtY = np.dot(np.transpose(Y), Y)
        double[:] initial_b = np.zeros(n_factors, dtype = np.float64)
        double[:, :] initial_A = YtY + reg * np.eye(n_factors, dtype = np.float64)
    
    with nogil, parallel(num_threads = n_threads):
        # allocate temp memory for each thread 
        b = <double*> malloc(sizeof(double) * n_factors)
        A = <double*> malloc(sizeof(double) * n_factors * n_factors)
        try:
            for u in prange(n_users, schedule = 'guided'):
                memcpy(b, &initial_b[0], sizeof(double) * n_factors)
                memcpy(A, &initial_A[0, 0], sizeof(double) * n_factors * n_factors)

                for index in range( indptr[u], indptr[u + 1] ):
                    i = indices[index]
                    confidence = data[index] + 1
                    
                    # update b;
                    # b += confidence * Y[i]
                    axpy(&n_factors, &confidence, &Y[i, 0], &one, b, &one)
                    
                    # A += (confidence - 1) * np.outer(factor, factor)
                    # for j in range(n_factors):
                    #     temp = (confidence - 1) * Y[i, j]
                    #     axpy(&n_factors, &temp, &Y[i, 0], &one, A + j * n_factors, &one)
                    for j in range(n_factors):
                        temp = confidence - 1
                        outer(&n_factors, &n_factors, &temp, 
                              &Y[i, 0], &one, &Y[i, 0], &one, A, &n_factors)
                
                # solve for Ax = b
                # X[u] = np.linalg.solve(A, b)
                posv('U', &n_factors, &one, A, &n_factors, b, &n_factors, &error)
                memcpy(&X[u, 0], b, sizeof(double) * n_factors)

        finally:
            free(A)
            free(b)